In [7]:
import numpy as np
import csv
import pandas as pd
import time 
# Predict via the median number of plays.

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'

In [2]:
# Load the training data.
train_data = {}
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = int(row[2])
    
        if not user in train_data:
            train_data[user] = {}
        
        train_data[user][artist] = plays



In [ ]:
# Compute the global median.
plays_array = []
for user, user_data in train_data.iteritems():
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
global_median = np.median(np.array(plays_array))
print "global median:", global_median

In [3]:
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [ ]:
# cluster user based on nationality and age
user_df = pd.read_csv("profiles.csv")
user_df.head()

In [ ]:
user_df["sex"][user_df["sex"] == 'f'] = 0
user_df["sex"][user_df["sex"] == 'm'] = 1
user_df["sex"] = user_df["sex"].fillna(0)

In [ ]:
country_dict = {}
for idx, i in enumerate(user_df["country"].unique()):
    country_dict[i] = idx
user_df["country"] = [country_dict[i] for i in user_df["country"]]

In [ ]:
user_df.head(10)

In [ ]:
user_df["age"] = user_df["age"].fillna(user_df["age"].mean())
user_df["age"].max()

In [ ]:
cluster = {}
info = {}
for row in user_df.iterrows():
    user = row[1]["user"]
    sex = row[1]["sex"]
    nation = row[1]["country"]
    age = row[1]["age"]
    
    info[user] = (nation, sex, age)
    if nation not in cluster:
        cluster[nation] = {}
        cluster[nation][sex] = {}
        cluster[nation][sex] = set([])
        cluster[nation][sex].add(user)
    else:
        try:
            cluster[nation][sex].add(user)
        except KeyError:
            cluster[nation][sex] = set([])
            cluster[nation][sex].add(user)

In [ ]:
import time
#data cluster
start = time.time()
cluster = {}
info = {}
count = 0
for row in user_df.iterrows():
    #print count
    count += 1
    user = row[1]["user"]
    sex = row[1]["sex"]
    nation = row[1]["country"]
    age = row[1]["age"]
    
    data = train_data[user] # data is a dict
    
    info[user] = (nation, sex, age)
    
    if nation not in cluster:
        cluster[nation] = {}
        cluster[nation][sex] = {}
        for artist in data:
            cluster[nation][sex][artist] = [data[artist]]
        cluster[nation][sex]["undefined"] = [user_medians[user]]
    else:
        if sex in cluster[nation]:
            for artist in data:
                try:
                    cluster[nation][sex][artist].append(data[artist])
                except KeyError:
                    cluster[nation][sex][artist] = [data[artist]]
            cluster[nation][sex]["undefined"].append(user_medians[user])
        else:
            cluster[nation][sex] = {}
            for artist in data:
                cluster[nation][sex][artist] = [data[artist]]
            try:
                cluster[nation][sex]["undefined"].append(user_medians[user])
            except KeyError:
                cluster[nation][sex]["undefined"] = [user_medians[user]]
print time.time() - start

In [ ]:
#cluster based on nation
import time
#data cluster
start = time.time()
cluster = {}
info = {}
count = 0
for row in user_df.iterrows():
    #print count
    count += 1
    user = row[1]["user"]
    sex = row[1]["sex"]
    nation = row[1]["country"]
    age = row[1]["age"]
    
    data = train_data[user] # data is a dict
    
    info[user] = (nation, sex, age)
    
    if nation not in cluster:
        cluster[nation] = {}
        for artist in data:
            cluster[nation][artist] = [data[artist]]
        cluster[nation]["undefined"] = [user_medians[user]]
    else:
        for artist in data:
            try:
                cluster[nation][artist].append(data[artist])
            except KeyError:
                cluster[nation][artist] = [data[artist]]
        cluster[nation]["undefined"].append(user_medians[user])
print time.time() - start

In [ ]:
len(cluster["China"])

In [ ]:
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open("test_groups_mean.csv", 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])
        #count = 0 
        for row in test_csv:
            count += 1
            #print count
            id     = row[0]
            user   = row[1]
            artist = row[2]
             
            nation = info[user][0]
            sex = info[user][1]
            
            #list_neigbor = cluster[nation][sex]
            try :
                value = cluster[nation][sex][artist]
            except KeyError:
                value = user_medians[user]
            #value = []
#             for i in list_neigbor:
#                 try:
#                     value.append(train_data[i][artist])
#                 except KeyError:
#                     value.append(user_medians[i])
            avg = np.mean(value)
            soln_csv.writerow([id,avg])
            #print avg
            #if user in user_medians:
                #soln_csv.writerow([id, user_medians[user]])
            #else:
                #print "User", id, "not in training data."
                #soln_csv.writerow([id, global_median])
#             if len(value) != 0:
#                 soln_csv.writerow([id, avg])
#             else:
#                 soln_csv.writerow([id, global_median])
            
            


In [ ]:
# write to test basd on nataion
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open("test_nation_median.csv", 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])
        #count = 0 
        for row in test_csv:
            count += 1
            #print count
            id     = row[0]
            user   = row[1]
            artist = row[2]
             
            nation = info[user][0]
            
            #list_neigbor = cluster[nation][sex]
#             try :
#                 value = cluster[nation][artist]
#             except KeyError:
#                 value = cluster[nation]["undefined"]
#             avg =int(np.median(value))
#             soln_csv.writerow([id,avg])
            if user in user_medians:
                soln_csv.writerow([id, user_medians[user]])
            else:
                print "User", id, "not in training data."
                soln_csv.writerow([id, global_median])
            



In [ ]:
user_df.head(20)

In [5]:
# Write out test solutions.
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open("user_median_0.97.csv", 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            if user in user_medians:
                soln_csv.writerow([id, 0.97 * user_medians[user]])
            else:
                print "User", id, "not in training data."
                soln_csv.writerow([id, global_median])


In [8]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm

# train_data = {}
# with open(train_file, 'r') as train_fh:
#     train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
#     next(train_csv, None)
#     for row in train_csv:
#         user   = row[0]
#         artist = row[1]
#         plays  = int(row[2])
    
#         if not user in train_data:
#             train_data[user] = {}
        
#         train_data[user][artist] = plays

# # Read in data

def loadData(filename, train = True):
    data = []
    y = []
    users=set()
    items=set()
    with open(filename, 'r') as f:
        data_csv = csv.reader(f, delimiter=',', quotechar='"')
        next(data_csv, None)
        if train:
            for row in data_csv:
                user   = row[0]
                artist = row[1]
                plays  = int(row[2])
            
                data.append({ "user_id": str(user), "movie_id": str(artist)})
                y.append(float(plays))
                users.add(user)
                items.add(artist)
        else:
            for row in data_csv:
                user   = row[0]
                artist = row[1]
            
                data.append({ "user_id": str(user), "movie_id": str(artist)})
                users.add(user)
                items.add(artist)

    return (data, np.array(y), users, items)

(train_data, y_train, train_users, train_items) = loadData(train_file, True)
(test_data, y_test, test_users, test_items) = loadData(test_file, False)
v = DictVectorizer()
X_train = v.fit_transform(train_data)
X_test = v.transform(test_data)

# Build and train a Factorization Machine
#fm = pylibfm.FM(num_factors=10, num_iter=100, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")

#fm.fit(X_train,y_train)

KeyboardInterrupt: 

In [12]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm

In [9]:
def loadData(filename, train = True):
    data = []
    y = []
    users=set()
    items=set()
    with open(filename) as f:
        for line in f:
            (user, artist, plays) = line.strip().split(',')
            if user == 'user':
                continue
            else:
                data.append({"user_id" : str(user), "artist" : str(artist)})
                y.append(float(plays))
                users.add(user)
                items.add(artist)
    return (data, np.array(y), users, items)

In [13]:
(train_data, y_train, train_users, train_items) = loadData(train_file, True)
v = DictVectorizer()
X_train = v.fit_transform(train_data)





In [ ]:
fm_1 = pylibfm.FM(num_factors=5, num_iter=100, verbose=True, task="regression", initial_learning_rate= 10 **(-6), learning_rate_schedule="optimal")

fm_1.fit(X_train,y_train)

In [ ]:
fm_2 = pylibfm.FM(num_factors=100, num_iter=200, verbose=True,validation_size = 0.02, task="regression", initial_learning_rate= 3 * 10 **(-6), learning_rate_schedule="optimal")

fm_2.fit(X_train,y_train)

Creating validation dataset of 0.02 of training for adaptive regularization
-- Epoch 1

In [ ]:
fm_3 = pylibfm.FM(num_factors=40, num_iter=300, verbose=True, task="regression", initial_learning_rate= 10 **(-6), learning_rate_schedule="optimal")

fm_3.fit(X_train,y_train)

In [ ]:
# write to test basd on nataion
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open("test_nation_median.csv", 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])
        #count = 0 
        for row in test_csv:
            count += 1
            #print count
            id     = row[0]
            user   = row[1]
            artist = row[2]
             
            nation = info[user][0]
            
            #list_neigbor = cluster[nation][sex]
#             try :
#                 value = cluster[nation][artist]
#             except KeyError:
#                 value = cluster[nation]["undefined"]
#             avg =int(np.median(value))
#             soln_csv.writerow([id,avg])
            if user in user_medians:
                soln_csv.writerow([id, user_medians[user]])
            else:
                print "User", id, "not in training data."
                soln_csv.writerow([id, global_median])